### sistema de recomendacion utulizando el the moviedb id

In [1]:
import pandas as pd
import numpy as np


from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [2]:
links = pd.read_csv('./data/new_data/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
movie = pd.read_csv('./data/new_data/movies.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating = pd.read_csv('./data/new_data/ratings.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#### Igualición de id en movies para utilizar los the movie database

In [5]:
movie = pd.merge(movie,links,how='inner',on='movieId')
movie = movie[['tmdbId','title']]
movie.dropna(subset=['tmdbId'],inplace=True)
movie.drop_duplicates(subset=['tmdbId'],inplace=True)
movie.columns = ['movie_id','title']
movie.head()

,movie_id,title
0,862.0,Toy Story (1995)
1,8844.0,Jumanji (1995)
2,15602.0,Grumpier Old Men (1995)
3,31357.0,Waiting to Exhale (1995)
4,11862.0,Father of the Bride Part II (1995)


#### Igualación de id en rating para utilizar los the movie database

In [6]:
rating = pd.merge(rating,links,how='inner',on='movieId')
rating = rating[['userId','tmdbId','rating']]
rating.dropna(subset=['tmdbId'],inplace=True)
rating.columns = ['user_id','movie_id','rating']
rating.head()

,user_id,movie_id,rating
0,1,862.0,4.0
1,5,862.0,4.0
2,7,862.0,4.5
3,15,862.0,2.5
4,17,862.0,4.5


### Pre procesamiento

Un proceso muy importante luego de haber leido la información, es proceder a mezclar nuestros dos dataset, con el objetivo de tener en un solo conjunto de datos toda la información correspondiente al nombre de la pelicula y las calificaciones que los usuarios han ido realizando a lo largo del tiempo.

El concepto de esta mezcla tiene mucho que ver con las bases de datos, en este caso estamos mezclando los dos dataset utilizando el movie_id como clave principal de union.

Luego de ello el objetivo es tener el promedio de calificación de la pelicula tomando en cuenta todas las calificaciones de distintos usuarios a esa misma pelicula.

In [7]:
dataset = pd.merge(rating, movie, how='inner', on='movie_id')
dataset.head()

,user_id,movie_id,rating,title
0,1,862.0,4.0,Toy Story (1995)
1,5,862.0,4.0,Toy Story (1995)
2,7,862.0,4.5,Toy Story (1995)
3,15,862.0,2.5,Toy Story (1995)
4,17,862.0,4.5,Toy Story (1995)


In [8]:
clean_dataset = dataset.groupby(by=['user_id','title'], as_index=False).agg({"rating":"mean"})
clean_dataset.head()

,user_id,title,rating
0,1,"13th Warrior, The (1999)",4.0
1,1,20 Dates (1998),4.0
2,1,"Abyss, The (1989)",4.0
3,1,"Adventures of Robin Hood, The (1938)",5.0
4,1,Alice in Wonderland (1951),5.0


### Proceso de recomendación utilizando el algoritmo KNN con la entrada del nombre de la película y el número de películas que se quieren recomendar:

Como primer paso tenemos que realizar el proceso de crear una matriz inversa, es decir que los titulos de las peliculas sean tomadas como filas, mientras que las columnas sean los id del usuario y la intersección entre la columna y fila sea la calificación del usuario en esa pelicula.

In [9]:
movie_to_user_df = clean_dataset.pivot(
     index='title',
   columns='user_id',
      values='rating').fillna(0)

movie_to_user_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


 Una de las desventajas de esta matriz, es que esta llena de valores 0 o nulos, este problema se lo conoce como "problema de esparcimiento" y esto ocasiona que las predicciones o recomendaciones que un sistema de recomendación desee hacer sean imprecisas debidos a estos valores basura, por tanto la solución es crear una matriz conocida como sparse, que soluciona este problema, a continuación realizaremos dicho proceso.

In [10]:
movie_to_user_sparse_df = csr_matrix(movie_to_user_df.values)
movie_to_user_sparse_df

<9713x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100819 stored elements in Compressed Sparse Row format>

Una vez realizado la matriz de esparcimiento, que nos ayuda a evitiar el problema sparse, procedemos a crear una lista de todas las peliculas que se tienen en el dataset, con el objetivo de darles un indice posteriormente, transformadolas en un diccionario que nos servira para después poder realizar las recomendaciones correspondientes.

In [11]:
movies_list = list(movie_to_user_df.index)
movies_list[:10]

["'71 (2014)",
 "'Hellboy': The Seeds of Creation (2004)",
 "'Round Midnight (1986)",
 "'Salem's Lot (2004)",
 "'Til There Was You (1997)",
 "'Tis the Season for Love (2015)",
 "'burbs, The (1989)",
 "'night Mother (1986)",
 '(500) Days of Summer (2009)',
 '*batteries not included (1987)']

In [12]:
movie_dict = {movie : index for index, movie in enumerate(movies_list)}
print(movie_dict)

{"'71 (2014)": 0, "'Hellboy': The Seeds of Creation (2004)": 1, "'Round Midnight (1986)": 2, "'Salem's Lot (2004)": 3, "'Til There Was You (1997)": 4, "'Tis the Season for Love (2015)": 5, "'burbs, The (1989)": 6, "'night Mother (1986)": 7, '(500) Days of Summer (2009)': 8, '*batteries not included (1987)': 9, '...All the Marbles (1981)': 10, '...And Justice for All (1979)': 11, '00 Schneider - Jagd auf Nihil Baxter (1994)': 12, '1-900 (06) (1994)': 13, '10 (1979)': 14, '10 Cent Pistol (2015)': 15, '10 Cloverfield Lane (2016)': 16, '10 Items or Less (2006)': 17, '10 Things I Hate About You (1999)': 18, '10 Years (2011)': 19, '10,000 BC (2008)': 20, '100 Girls (2000)': 21, '100 Streets (2016)': 22, '101 Dalmatians (1996)': 23, '101 Dalmatians (One Hundred and One Dalmatians) (1961)': 24, "101 Dalmatians II: Patch's London Adventure (2003)": 25, '101 Reykjavik (101 Reykjavík) (2000)': 26, '102 Dalmatians (2000)': 27, '10th & Wolf (2006)': 28, '10th Kingdom, The (2000)': 29, '10th Victim,

Posterior a ello, procedemos a **crear nuestro modelo de KNN** utilizando la metrica de similitud de cosenos y un algoritmo bruto para conseguir una recomendación a corde al conjunto de datos que se tienen actualmente. De igual forma con el modelo ya creado procedemos a realizar un entrenamiento con la finalidad de que nuestro modelo aprenda del conjunto de datos que se tienen.

In [13]:
knn_movie_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_movie_model.fit(movie_to_user_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine')

Una vez entrenado nuestro modelo KNN, procedemos a realizar la recomendación de una pelicula en particular, utilizando el nombre como lo hemos indicado al inicio de nuestro modelo. Con ello creamos un método o función que toma como entrada el nombre de la película y el número de similitudes que queremos obtener con el nombre de la pelicula principal.

In [14]:
def get_similar_movies(movie, n = 10):
    index = movie_dict[movie]
    knn_input = np.asarray([movie_to_user_df.values[index]])
    n = min(len(movies_list)-1,n)
    distances, indices = knn_movie_model.kneighbors(knn_input, n_neighbors=n+1)
  
    print("Top",n,"peliculas que tienen mucha similitud con la pelicula-",movie, "son: ")
    print(" ")
    for i in range(1,len(distances[0])):
        print(movies_list[indices[0][i]])

In [15]:
movie_name = 'Toy Story (1995)'

get_similar_movies(movie_name,15)

Top 15 peliculas que tienen mucha similitud con la pelicula- Toy Story (1995) son: 
 
Toy Story 2 (1999)
Jurassic Park (1993)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Lion King, The (1994)
Star Wars: Episode VI - Return of the Jedi (1983)
Mission: Impossible (1996)
Groundhog Day (1993)
Back to the Future (1985)
Shrek (2001)
Aladdin (1992)
Apollo 13 (1995)
Pulp Fiction (1994)
Star Wars: Episode V - The Empire Strikes Back (1980)


<div class="alert alert-success">
    <b>Con ello el sistema de recomendación funciona correctamente!!</b> <br>
    <br>- El modelo toma como entrada el nombre de la película y el número de películas que se desea recomendar.
    <br>- El modelo devolvera películas relacionadas a la película que se ingreso para recomendar.
    <br>- Ahora se indica el proceso para proceder a guardar el modelo en un formato <b>.pkl</b> para utilizarlo en un servicio web. 
</div>

### Proceso que permite guardar el modelo KNN en un formato .pkl

Una de los procesos más importantes después de realizar un modelo de ML es realizar la exportación de dicho modelo, esto debido a que muchas veces estos modelos suelen quedarse en cuadernos jupyter por lo que solo quedan en desarrollo para pruebas. Lo importante es implementar este modelo en un sistema para ver su funcionamiento.

A continuación se presentan los pasos para exportar el modelo en un formato .pkl

#### Joblib para exportar y cargar modelos

Joblib es una libreria que permite facilitar la creación de pipelines con python, sin embargo, esta librería también funciona para poder exportar e importar modelos para poder realizar las pruebas del mismo.

In [16]:
import joblib

#### exportando modelo 

Exportar un modelo de ml es muy sencillo con esta herramienta, solamente debemos utilizar la función **dump** enviando como parametros nuestro modelo de ML y el path donde se va a ubicar nuestro modelo con formato .pkl

In [17]:
name_model = 'knn_model'
path_save = './model/{}.pkl'.format(name_model)
joblib.dump(knn_movie_model, path_save)

['./model/knn_model.pkl']

### importando un modelo 

Ahora, imaginemos que tenemos que consumir un modelo en un servicio web como por ejemplo en django, pues joblib permite realizar de igual forma la importación de este modelo y tratarlo como servicio. A continuación se indica como se debe importar el modelo.

In [18]:
name_model_load = 'knn_model'
path_load = './model/{}.pkl'.format(name_model_load)
model_load = joblib.load(path_load,mmap_mode='r')

Con el modelo de ML importado, en el siguiente ejemplo se simula un método para obtener las recomendaciones del sistema de recomendación utilizando el modelo importado. Con ello obtenemos el siguiente resultado.

In [19]:
def get_similar_movies_with_model_load(movie, n = 10):
    index = movie_dict[movie]
    knn_input = np.asarray([movie_to_user_df.values[index]])
    n = min(len(movies_list)-1,n)
    distances, indices = model_load.kneighbors(knn_input, n_neighbors=n+1)
  
    print("Top",n,"peliculas que tienen mucha similitud con la pelicula-",movie, "son: ")
    print(" ")
    for i in range(1,len(distances[0])):
        print(movies_list[indices[0][i]])

In [20]:
movie_name = 'Toy Story (1995)'

get_similar_movies_with_model_load(movie_name,15)

Top 15 peliculas que tienen mucha similitud con la pelicula- Toy Story (1995) son: 
 
Toy Story 2 (1999)
Jurassic Park (1993)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Lion King, The (1994)
Star Wars: Episode VI - Return of the Jedi (1983)
Mission: Impossible (1996)
Groundhog Day (1993)
Back to the Future (1985)
Shrek (2001)
Aladdin (1992)
Apollo 13 (1995)
Pulp Fiction (1994)
Star Wars: Episode V - The Empire Strikes Back (1980)


<div class="alert alert-success">
    <b>Con ello se puede utilizar el modelo de ML en un servicio web!!!</b> <br>
    <br>- Se aprendio a crear un modelo de ML utilizando el algoritmo KNN.
    <br>- Se aprendio sobre problemas de esparcimiento y como resolverlo.
    <br>- Se aprendio como exportar e importar el modelo de ML para usarlo en un servicio web.
</div>

In [21]:
movie_to_user_df

user_id,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,2.0
xXx: State of the Union (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5
